# Complete Silver Layer
Historisez dans le lakehouse en silver toutes les tables importées précédemment dans la couche bronze.

### Connecting to the Silver Layer (Target)

In [0]:
USE CATALOG cab_lakehouse;
USE DATABASE silver;

In [0]:
DECLARE OR REPLACE load_date = current_timestamp();
VALUES load_date;

### Incremental Load

In [0]:
-- load custormeraddress

MERGE INTO silver.customeraddress AS tgt
USING (
  SELECT 
    CustomerID	    AS customer_id,
    AddressID	      AS address_id,
    AddressType	    AS address_type, 
    rowguid	        AS rowguid, 
    ModifiedDate	  AS modified_date
  FROM bronze.customeraddress
) AS src
ON tgt.customer_id = src.customer_id 
  AND tgt._tf_valid_to IS NULL --Only match against 'active' records in silver
WHEN MATCHED AND (
        tgt.address_id	 != src.address_id,
    OR tgt.address_type	 != src.address_type,
    OR tgt.rowguid	 != src.rowguid, 
    OR tgt.modified_date	 != src.modified_date
   -- etc. for any columns you want to track changes on
) AND tgt._tf_valid_to IS NULL THEN
  -- 1) Close the old record by setting _tf_valid_to
  UPDATE SET 
    tgt._tf_valid_to    = load_date,
    tgt._tf_update_date = load_date
WHEN NOT MATCHED BY SOURCE AND tgt._tf_valid_to IS NULL THEN
  -- 2) Close the deleted record by setting _tf_valid_to
  UPDATE SET 
    tgt._tf_valid_to    = load_date,
    tgt._tf_update_date = load_date
;


In [0]:

--SELECT column_name FROM information_schema.columns WHERE table_name = 'customeraddress'  

In [0]:
-- load productDescription

In [0]:
-- load productmodel


In [0]:
-- load productmodelproductdescription